# Run the following cell to install the required dependencies
###### **NOTE:** This step is compulsory

In [ ]:
#@title Install the required dependencies
!pip install spotipy
!pip install pafy
!pip install youtube_title_parse
!pip install youtube-dl
!pip install urllib3>=1.26
!pip install requests>=2.25
!pip install pytube
!apt install ffmpeg
exit()

#Download Youtube/Spotify playlist/album!
######**NOTE:** For Youtube Playlist, make sure the playlist is public

In [ ]:
%reset -f

import json, base64, os, requests, time, sys, subprocess, pafy, spotipy, zipfile
from spotipy.oauth2 import SpotifyClientCredentials
import os, urllib.request, re, subprocess, sys, pafy
from youtube_title_parse import get_artist_title
from urllib.parse import quote
from zipfile import ZipFile
from pytube import Playlist
from google.colab import files

class common():

    def __init__(self, spo=False):
        self.spo = spo

    def get_url(self,s):
        query = "https://www.youtube.com/results?search_query="+quote(s)
        baseurl = "https://www.youtube.com/watch?v="
        response = urllib.request.urlopen(query)
        html = response.read()
        video_ids = re.findall(r"watch\?v=(\S{11})", html.decode())
        urls = []
        if self.spo:
            for i in video_ids:
                try:
                    pafy.new(baseurl+i)
                    urls.append(baseurl+i)
                    return urls
                except:
                    continue
        else:
            j = 0
            for i in video_ids:
                try:
                    pafy.new(baseurl+i)
                    urls.append(baseurl+i)
                    j+=1
                    if j==7:
                        break
                except:
                    continue
            return urls

    def convert(self, old, new, alburl, flac=False):        
        if self.spo:
            if not flac:
              !ffmpeg -hide_banner -loglevel quiet -i '$old' -b:a 320k 'w$new'
              os.remove(old)
              urllib.request.urlretrieve(alburl, "thumb.png")
              print("Adding album art")
              !ffmpeg -hide_banner -loglevel quiet -i 'w$new' -i thumb.png -map 0:0 -map 1:0 -codec copy -id3v2_version 3 -metadata:s:v title="Album cover" -metadata:s:v comment="Cover (front)" '$new'
              os.remove("w"+new)
              os.remove("thumb.png")
        else:
            if not flac:
              !ffmpeg -hide_banner -loglevel quiet -i '$old' -b:a 320k '$new'
            os.remove(old)

    # Download the song
    def download_song(self,url, albart, sponame, z):

        try:
            v = pafy.new(url)
        except Exception as e:
            print(f"\nSome error occurred while fetching the details of the song : {e}\n")
            return

        name = v.title
        audio = v.getbestaudio()
        print(f"Downloading {name}")
        audio.download()
        dirs = os.listdir()
        try:

            if self.spo:
                artist, title = get_artist_title(sponame)   
            else:
                artist, title = get_artist_title(name)
            for i in dirs:
                    if name in i:
                        track_name = title+" - "+artist+".mp3"
                        track_name = track_name.replace("\\"," ").replace("/"," ").replace(":"," ").replace("*"," ").replace("?"," ").replace("\""," ").replace("<"," ").replace(">"," ").replace("|"," ")
                        print(f"Converting the audio format to mp3")
                        self.convert(i, track_name, albart)

            for i in os.listdir():
                if "_" in i:
                    try:
                        os.rename(i,i.replace("_"," "))
                    except FileExistsError:
                        try:
                            os.rename(i,i.replace("_","  "))
                        except FileExistsError:
                            pass
        except TypeError:
            for i in dirs:
                if name in i:
                    ind1 = len(i) - 1 - i[::-1].index('.')
                    ext = i[ind1:]
                    if ext in i:
                        track_name = name[:ind1]+".mp3"
                        print(f"Converting the audio format to mp3")
                        self.convert(i, track_name, albart)
            for i in os.listdir():
                if "_" in i:
                    os.rename(i,i.replace("_"," "))

class Spotify():

  def __init__(self, link, plname):
        self.link = link
        self.plname = plname
        self.destination = "/content/sec.json"
        self.ytd = common(spo=True)

  def get_json(self):
      id = '1SFCB1mjcNz3U5X0HZTy4j5kpoMdemKWH'
      URL = "https://docs.google.com/uc?export=download"
      session = requests.Session()
      response = session.get(URL, params = { 'id' : id }, stream = True)
      token = None
      for key, value in response.cookies.items():
          if key.startswith('download_warning'):
              token = value
          else:
              token = None
      if token:
          params = { 'id' : id, 'confirm' : token }
          response = session.get(URL, params = params, stream = True)
      CHUNK_SIZE = 32768
      with open(self.destination, "wb") as f:
          for chunk in response.iter_content(CHUNK_SIZE):
              if chunk:
                  f.write(chunk)

  def get_credentials(self):
      empty = {}
      with open(self.destination, 'r') as f:
          data = json.load(f)
      with open(self.destination,'w') as f:
          json.dump(empty,f)
      for key in data.keys():
          data[key] = base64.b64decode(data[key]).decode('utf8')
      client_id = data['id']
      client_secret = data['secret']
      client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
      return spotipy.Spotify(client_credentials_manager=client_credentials_manager)

  def get_id(self,url):
      if "playlist" in url:
          ind = url.index("playlist")
          return url[ind+9:ind+31]
      elif "album" in url:
          ind = url.index("album")
          return url[ind+6:ind+28]

  def get_playlist_tracks(self, sp, plID):
      """ returns a dictionary of track name as key and artist name as value of a spotify playlist given the spotipy token credentials object and playlist id """

      alburl = []
      together = {}
      tracks = {}
      playlist = {}
      offset = 0
      total = sp.playlist_tracks(plID)['total']
      while total-offset>=0:
          playlist = sp.playlist_tracks(plID,offset=offset)
          for i in range(len(playlist['items'])):
              if len(playlist['items'][i]['track']['album']['images']) == 0:
                  together[playlist['items'][i]['track']['name']] = [playlist['items'][i]['track']['artists'][0]['name'],'https://virginradio.co.uk/sites/virginradio.co.uk/files/song_cover/20160626/coverart_1.png']
              else:
                  together[playlist['items'][i]['track']['name']] = [playlist['items'][i]['track']['artists'][0]['name'],playlist['items'][i]['track']['album']['images'][1]['url']]
          offset +=100
      for key in together.keys():
          tracks[key] = together[key][0]
          alburl.append(together[key][1])
      return tracks, alburl

  def get_album_tracks(self, sp, alID):
      """ returns a dictionary of track name as key and artist name as value of a spotify album given the spotipy token credentials object and album id """

      album = sp.album_tracks(alID)
      tracks = {}
      alburl = []
      for i in range(len(album['items'])):
          tracks[album['items'][i]['name']] = album['items'][i]['artists'][0]['name']
          alburl.append(sp.track(album['items'][i]['id'])['album']['images'][1]['url'])
      return tracks, alburl

  def get_yt_urls(self,tracks):
      urls = []
      for key in tracks.keys():
          try:
              got_url = (key+"+"+tracks[key]).replace(' ','+')
              print(f"Fetching the details of {key} - {tracks[key]}")
              urls.append(self.ytd.get_url(got_url)[0])
          except Exception as e:
              print(f"\nCould not fetch the link for \"{key} {tracks[key]}\" because: {e}\nmaybe download it as a single")
      return urls

  def download_PL(self,plname, urls, alburl, sponame):
    
    try:
      os.mkdir(plname)
      os.chdir(plname)
    except FileExistsError:
      !rm -rf '$self.plname'
      os.mkdir(plname)
      os.chdir(plname)

    c=1
    for i,j,k in zip(urls, alburl, sponame):
        self.ytd.download_song(i, j, k, '')
        c+=1
  
  def spodl(self):
      self.get_json()
      sponame = []
      sp = self.get_credentials()
      if "playlist" in self.link:    
          plID = self.get_id(self.link)
          tracks,alburls = self.get_playlist_tracks(sp, plID)
      elif "album" in self.link:
          alID = self.get_id(self.link)
          tracks, alburls = self.get_album_tracks(sp,alID)
      else: 
          print("Invalid link, exiting")
          return
      start_time = time.time()
      print("\nFetching all the relevant URLs\n")
      urls = self.get_yt_urls(tracks)
      end_time = time.time()
      print(f"Time taken to fetch the URLs from Youtube: %.2f secs\n"%(end_time-start_time))
      print("\nDownloading the songs\n")
      for i in tracks.keys():
          sponame.append(tracks[i]+" - "+i)
      self.download_PL(self.plname, urls, alburls, sponame)
      os.chdir('..')

class Youtube:
  
  def __init__(self, link, plName):
    self.plLink = link
    self.plName = plName

  def get_playlist_url(self,plLink):
    pl_links = []
    pl = Playlist(plLink)
    for url in pl.video_urls:
        pl_links.append(url)
    return pl_links
  
  def download_playlist(self):
    cm = common()
    try:
      os.mkdir(self.plName)
      os.chdir(self.plName)
    except FileExistsError:
      !rm -rf '$self.plName'
      os.mkdir(self.plName)
      os.chdir(self.plName)

    if "https://www" in self.plLink:
        self.plLink = self.plLink.replace("https://www","https://music")
    
    start_time = time.time()
    try:
        plLinks = self.get_playlist_url(self.plLink)
    except Exception as e:
        print(f"Something went wrong. Maybe check your URL. Here's the reason from the compiler: {e}")
        print("Exiting the program")
        return
    end_time = time.time()
    print(f"\nTime taken to fetch the URLs from Youtube: %.2f secs\n"%(end_time-start_time))
    
    for i in plLinks:
        cm.download_song(i,"",'','')
    os.chdir('..')

def zipper(pl):
  if f'/content/{pl}.zip' in os.listdir():
    !rm -rf '/content/$pl''.zip'
  file_paths = []
  for root, directories, files in os.walk(f'/content/{pl}'):
      for filename in files:
          filepath = os.path.join(root,filename)
          file_paths.append(filepath)
  with ZipFile(f'/content/{pl}.zip','w') as zip:
        for file in file_paths:
            zip.write(file,compress_type=zipfile.ZIP_DEFLATED)

if __name__=='__main__':
  
  os.chdir('/content')
  Link = "https://open.spotify.com/playlist/6joKh5OlOlg6JRzqhwbel1" #@param {type:"string"}
  Playlist_Name = "My Playlist" #@param {type:"string"}
  if 'spotify' in Link:
    Spotify(Link, Playlist_Name).spodl()
    os.remove("/content/sec.json")
    zipper(Playlist_Name)
    files.download(Playlist_Name+".zip") 
  elif 'youtube' in Link:
    Youtube(Link, Playlist_Name).download_playlist()
    zipper(Playlist_Name)
    files.download(Playlist_Name+".zip") 
  else:
    print("Invalid URL")